<a href="https://colab.research.google.com/github/kuostar0620-jpg/114-1KUO-REPO-/blob/main/41371124Hweek2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

本功能會列出
=== 支出追蹤器選單 ===
1. 記錄新的支出
2. 查看最新一筆記錄
3. 顯示所有記錄表格與統計
4. 匯出 CSV 檔案
5. 刪除單筆記錄
6. 離開程式

含以上這些功能，提供方便的記帳模式


In [ ]:
import pandas as pd
import os
from datetime import datetime
import pytz

# --- 核心功能: 判斷餐別 ---
def get_meal_type(time_str):
    """
    根據時間字串判斷餐別。
    """
    try:
        time_obj = datetime.strptime(time_str, '%H:%M:%S').time()

        breakfast_start = datetime.strptime('00:00:00', '%H:%M:%S').time()
        breakfast_end = datetime.strptime('11:59:59', '%H:%M:%S').time()
        lunch_start = datetime.strptime('12:00:00', '%H:%M:%S').time()
        lunch_end = datetime.strptime('15:59:59', '%H:%M:%S').time()
        dinner_start = datetime.strptime('16:00:00', '%H:%M:%S').time()
        dinner_end = datetime.strptime('23:59:59', '%H:%M:%S').time()

        if breakfast_start <= time_obj <= breakfast_end:
            return "早餐"
        elif lunch_start <= time_obj <= lunch_end:
            return "午餐"
        elif dinner_start <= time_obj <= dinner_end:
            return "晚餐"
        else:
            return "其他"

    except (ValueError, IndexError):
        return "未知"

# --- 功能 1: 記錄新的支出 ---
def record_new_entry():
    """
    讓使用者輸入並記錄一筆新的支出。
    """
    try:
        taipei_tz = pytz.timezone('Asia/Taipei')
        now_taipei = datetime.now(taipei_tz)

        item = input("請輸入品項：")

        amount_str = input("請輸入金額：")
        try:
            amount = float(amount_str)
        except ValueError:
            print("---")
            print("金額輸入無效，請輸入數字。本次記錄失敗。")
            return

        date_to_write = input(f"請輸入日期 (例如: {now_taipei.strftime('%Y-%m-%d')})，如果留空將使用當前日期：")
        if not date_to_write:
            date_to_write = now_taipei.strftime('%Y-%m-%d')

        time_to_write = input(f"請輸入時間 (例如: {now_taipei.strftime('%H:%M:%S')})，如果留空將使用當前時間：")
        if not time_to_write:
            time_to_write = now_taipei.strftime('%H:%M:%S')

        meal_type = get_meal_type(time_to_write)

        log_entry = f"日期: {date_to_write}, 時間: {time_to_write}, 品項: {item}, 金額: {amount:.2f}, 餐別: {meal_type}\n"

        with open("log.txt", "a", encoding="utf-8") as file:
            file.write(log_entry)

        print("---")
        print("記錄成功！")
        print("---")

    except Exception as e:
        print(f"發生錯誤：{e}")

# --- 功能 2: 查看最新記錄 ---
def view_latest_log():
    """
    讀取 log.txt 檔案並顯示最後一筆記錄。
    """
    if not os.path.exists("log.txt"):
        print("---")
        print("目前沒有任何記錄檔案。")
        print("---")
        return

    try:
        with open("log.txt", "r", encoding="utf-8") as file:
            lines = file.readlines()
            if not lines:
                print("---")
                print("記錄檔案是空的。")
                print("---")
                return

            latest_entry = lines[-1].strip()
            print("---")
            print("最新的記錄如下：")
            print(latest_entry)
            print("---")

    except Exception as e:
        print(f"讀取檔案時發生錯誤：{e}")

# --- 功能 3: 讀取並顯示表格 ---
def read_log_to_table(file_path="log.txt"):
    """
    讀取 log.txt 檔案，並將資料轉換成一個 Pandas DataFrame 表格。
    """
    if not os.path.exists(file_path):
        return None

    data_list = []
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            for line in file:
                line = line.strip()
                if not line:
                    continue
                parts = line.split(", ")
                entry = {}
                for part in parts:
                    key_value = part.split(": ", 1)
                    if len(key_value) == 2:
                        key = key_value[0]
                        value = key_value[1]
                        entry[key] = value
                if entry:
                    data_list.append(entry)

        if not data_list:
            return None

        df = pd.DataFrame(data_list)

        if '金額' in df.columns:
            df['金額'] = pd.to_numeric(df['金額'], errors='coerce')

        return df

    except Exception as e:
        return None

def view_all_records():
    """
    顯示所有記錄的表格，並進行簡單的統計。
    """
    df = read_log_to_table()

    if df is not None:
        print("---")
        print("以下是您的所有記錄表格：")
        print("---")
        print(df)
        print("---")

        if '金額' in df.columns:
            total_amount = df['金額'].sum()
            print(f"總金額：{total_amount:.2f}")
            print("---")

            if '餐別' in df.columns:
                print("各餐別的總金額：")
                meal_summary = df.groupby('餐別')['金額'].sum().to_string()
                print(meal_summary)
                print("---")
    else:
        print("---")
        print("目前沒有任何記錄或檔案格式不正確。")
        print("---")

# --- 功能 4: 匯出 CSV 檔案 ---
def export_to_csv():
    """
    將記錄檔案轉換為 CSV 格式，並匯出為 log.csv。
    """
    df = read_log_to_table()

    if df is None:
        print("---")
        print("目前沒有任何記錄或檔案格式不正確，無法匯出。")
        print("---")
        return

    try:
        df.to_csv("log.csv", index=False, encoding="utf-8-sig")
        print("---")
        print("所有記錄已成功匯出至 log.csv 檔案！")
        print("您可以將此檔案匯入至 Google 試算表或其他試算表軟體。")
        print("---")

    except Exception as e:
        print(f"匯出檔案時發生錯誤：{e}")

# --- 功能 5: 刪除單筆記錄 ---
def delete_single_entry():
    """
    顯示所有記錄，並讓使用者選擇要刪除的單筆記錄。
    """
    if not os.path.exists("log.txt"):
        print("---")
        print("目前沒有任何記錄檔案，無法刪除。")
        print("---")
        return

    try:
        with open("log.txt", "r", encoding="utf-8") as file:
            lines = file.readlines()

        if not lines:
            print("---")
            print("記錄檔案是空的，沒有可刪除的記錄。")
            print("---")
            return

        print("---")
        print("以下是所有記錄：")
        for i, line in enumerate(lines):
            print(f"{i+1}. {line.strip()}")
        print("---")

        try:
            record_number = int(input("請輸入要刪除的記錄編號："))
            if 1 <= record_number <= len(lines):
                # 確認刪除
                confirmation = input(f"您確定要刪除第 {record_number} 筆記錄嗎？(y/n): ").lower()
                if confirmation == 'y':
                    # 刪除指定的行
                    del lines[record_number - 1]

                    # 將剩餘的行寫回檔案
                    with open("log.txt", "w", encoding="utf-8") as file:
                        file.writelines(lines)
                    print("---")
                    print(f"第 {record_number} 筆記錄已成功刪除。")
                    print("---")
                else:
                    print("---")
                    print("操作已取消。")
                    print("---")
            else:
                print("---")
                print("無效的記錄編號。")
                print("---")

        except ValueError:
            print("---")
            print("輸入無效，請輸入數字。")
            print("---")

    except Exception as e:
        print(f"處理檔案時發生錯誤：{e}")


# --- 主選單和程式執行 ---
def main_menu():
    """
    主選單，讓使用者選擇要執行的功能。
    """
    while True:
        print("\n=== 支出追蹤器選單 ===")
        print("1. 記錄新的支出")
        print("2. 查看最新一筆記錄")
        print("3. 顯示所有記錄表格與統計")
        print("4. 匯出 CSV 檔案")
        print("5. 刪除單筆記錄")
        print("6. 離開程式")

        choice = input("請輸入你的選擇 (1-6)：")

        if choice == '1':
            record_new_entry()
        elif choice == '2':
            view_latest_log()
        elif choice == '3':
            view_all_records()
        elif choice == '4':
            export_to_csv()
        elif choice == '5':
            delete_single_entry()
        elif choice == '6':
            print("程式已結束，感謝使用！")
            break
        else:
            print("---")
            print("無效的選擇，請輸入 1 到 6 之間的數字。")
            print("---")

if __name__ == "__main__":
    try:
        import pandas as pd
        import pytz
    except ImportError as e:
        print("錯誤：程式需要以下函式庫。請先安裝：")
        print(f"pip install {e.name}")
        exit()

    main_menu()


=== 支出追蹤器選單 ===
1. 記錄新的支出
2. 查看最新一筆記錄
3. 顯示所有記錄表格與統計
4. 匯出 CSV 檔案
5. 刪除單筆記錄
6. 離開程式


KeyboardInterrupt: Interrupted by user